# Lab 3 - N-Puzzle 

In [150]:
from typing import List, Tuple
from collections import namedtuple
from random import choice
import numpy as np
import heapq

In [ ]:
# Define the dimension of the puzzle (e.g., 3x3 grid)
PUZZLE_DIM = 3

In [152]:
def available_actions(state: np.ndarray) -> List[Tuple[int, int]]:
    """
    Determines all valid moves for the blank tile (0) in the given puzzle state.

    Parameters:
    - state (np.ndarray): The current configuration of the puzzle.

    Returns:
    - List[Tuple[int, int]]: A list of valid moves as (row_offset, col_offset).
    """
    row, col = np.where(state == 0)  # Locate the blank tile
    row, col = row[0], col[0]  # Unpack row and column values
    moves = []
    if row > 0: moves.append((-1, 0))  # Move blank up
    if row < state.shape[0] - 1: moves.append((1, 0))  # Move blank down
    if col > 0: moves.append((0, -1))  # Move blank left
    if col < state.shape[1] - 1: moves.append((0, 1))  # Move blank right
    return moves


def apply_action(state: np.ndarray, move: Tuple[int, int]) -> np.ndarray:
    """
    Executes a specified move, updating the puzzle state by swapping tiles.

    Parameters:
    - state (np.ndarray): The current configuration of the puzzle.
    - move (Tuple[int, int]): The move as (row_offset, col_offset).

    Returns:
    - np.ndarray: A new puzzle configuration after the move is applied.
    """
    row, col = np.where(state == 0)  # Locate the blank tile
    row, col = row[0], col[0]  # Unpack row and column values
    new_state = state.copy()
    new_row, new_col = row + move[0], col + move[1]  # Apply the move offsets
    # Swap the blank tile with the target tile
    new_state[row, col], new_state[new_row, new_col] = new_state[new_row, new_col], new_state[row, col]
    return new_state


def generate_solvable_puzzle(goal_state: np.ndarray, num_steps: int = 100000) -> np.ndarray:
    """
    Generates a randomized but solvable configuration of the puzzle by applying
    a series of valid moves starting from the goal state.

    Parameters:
    - goal_state (np.ndarray): The solved configuration of the puzzle.
    - num_moves (int): The number of random moves to apply.

    Returns:
    - np.ndarray: A randomized, solvable puzzle state.
    """
    # Start with the goal state
    current_state = goal_state.copy()
    
    # Apply the specified number of random valid moves
    for _ in range(num_steps):
        valid_moves = available_actions(current_state)  # Determine valid moves
        selected_move = choice(valid_moves)  # Randomly pick one move
        current_state = apply_action(current_state, selected_move)  # Apply the move
    
    # Return the randomized solvable state
    return current_state

    

def manhattan_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    """
    Calculates the Manhattan distance heuristic for the current state.

    Parameters:
    - state (np.ndarray): The current puzzle configuration.
    - goal_state (np.ndarray): The target puzzle configuration.

    Returns:
    - int: The sum of Manhattan distances for all tiles.
    """
    positions = {value: (x, y) for x, row in enumerate(goal_state) for y, value in enumerate(row)}
    total_distance = sum(
        abs(x - positions[value][0]) + abs(y - positions[value][1])
        for x, row in enumerate(state) for y, value in enumerate(row) if value != 0
    )
    return total_distance



In [153]:
def a_star(start_state: np.ndarray, goal_state: np.ndarray) -> Tuple[List[np.ndarray], int, int]:
    """
    Solves the sliding puzzle using the optimized A* algorithm.

    Parameters:
    - start_state (np.ndarray): The initial puzzle configuration.
    - goal_state (np.ndarray): The target puzzle configuration.

    Returns:
    - Tuple[List[np.ndarray], int, int]:
      - A list of states leading to the solution.
      - The number of moves to solve the puzzle.
      - The total number of explored nodes.
    """
    # Initialize the priority queue (frontier)
    # Each element is a tuple: (f_cost, binary_state, current_state)
    frontier = []
    heapq.heappush(frontier, (0, start_state.tobytes(), start_state))

    # Structures for tracking visited states and reconstructing the path
    explored = set()  # Keeps track of explored states to avoid revisiting
    parent_map = {start_state.tobytes(): None}  # Maps each state's binary representation to its predecessor
    g_costs = {start_state.tobytes(): 0}  # Tracks the actual cost (g(n)) to reach each state

    nodes_explored = 0  # Counter for the total number of nodes expanded

    while frontier:
        # Retrieve the state with the lowest f(n) from the priority queue
        f_cost, current_bytes, current_state = heapq.heappop(frontier)

        # Skip the state if it has already been explored
        if current_bytes in explored:
            continue

        # Check if the current state matches the goal state
        if np.array_equal(current_state, goal_state):
            # Reconstruct the solution path by backtracking through parent_map
            path = []
            while current_bytes is not None:
                path.append(current_state)
                current_bytes = parent_map[current_bytes]  # Move to the parent state
                if current_bytes:  # If a parent exists, reconstruct its NumPy array
                    current_state = np.frombuffer(current_bytes, dtype=start_state.dtype).reshape(start_state.shape)
            path.reverse()  # Reverse the path to get the correct order (start -> goal)
            return path, len(path) - 1, nodes_explored  # Return path, moves, and explored nodes

        # Mark the current state as explored
        explored.add(current_bytes)
        nodes_explored += 1

        # Generate all valid neighbors (states reachable in one move)
        for move in available_actions(current_state):
            # Apply the move to generate the next state
            next_state = apply_action(current_state, move)
            next_bytes = next_state.tobytes()  # Convert the state to its binary representation

            # Calculate g(n) for the neighbor
            tentative_g_cost = g_costs[current_bytes] + 1  # Increment cost by 1 for each move

            # If the state hasn't been visited or a better path is found
            if next_bytes not in g_costs or tentative_g_cost < g_costs[next_bytes]:
                g_costs[next_bytes] = tentative_g_cost  # Update the g(n) value
                h_cost = manhattan_distance(next_state, goal_state)  # Compute the heuristic (h(n))
                f_cost = tentative_g_cost + h_cost  # Total estimated cost f(n) = g(n) + h(n)
                heapq.heappush(frontier, (f_cost, next_bytes, next_state))  # Add the state to the frontier
                parent_map[next_bytes] = current_bytes  # Track its parent for path reconstruction

    # If the goal state is not reachable, return an empty solution
    return [], 0, nodes_explored


In [154]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
initial_state = generate_solvable_puzzle(goal_state)

# Solve and print the results
solution, moves, explored = a_star(initial_state, goal_state)
print(f"Solution Path ({moves} moves, {explored} nodes explored):")
for step, s in enumerate(solution):
    print(f"Step {step}:\n{s}\n")


Solution Path (20 moves, 309 nodes explored):
Step 0:
[[1 5 8]
 [4 6 2]
 [0 3 7]]

Step 1:
[[1 5 8]
 [0 6 2]
 [4 3 7]]

Step 2:
[[0 5 8]
 [1 6 2]
 [4 3 7]]

Step 3:
[[5 0 8]
 [1 6 2]
 [4 3 7]]

Step 4:
[[5 8 0]
 [1 6 2]
 [4 3 7]]

Step 5:
[[5 8 2]
 [1 6 0]
 [4 3 7]]

Step 6:
[[5 8 2]
 [1 0 6]
 [4 3 7]]

Step 7:
[[5 8 2]
 [1 3 6]
 [4 0 7]]

Step 8:
[[5 8 2]
 [1 3 6]
 [4 7 0]]

Step 9:
[[5 8 2]
 [1 3 0]
 [4 7 6]]

Step 10:
[[5 8 2]
 [1 0 3]
 [4 7 6]]

Step 11:
[[5 0 2]
 [1 8 3]
 [4 7 6]]

Step 12:
[[0 5 2]
 [1 8 3]
 [4 7 6]]

Step 13:
[[1 5 2]
 [0 8 3]
 [4 7 6]]

Step 14:
[[1 5 2]
 [4 8 3]
 [0 7 6]]

Step 15:
[[1 5 2]
 [4 8 3]
 [7 0 6]]

Step 16:
[[1 5 2]
 [4 0 3]
 [7 8 6]]

Step 17:
[[1 0 2]
 [4 5 3]
 [7 8 6]]

Step 18:
[[1 2 0]
 [4 5 3]
 [7 8 6]]

Step 19:
[[1 2 3]
 [4 5 0]
 [7 8 6]]

Step 20:
[[1 2 3]
 [4 5 6]
 [7 8 0]]

